<br>
<center> <font size="7" color="#0086DD" aling="center">Development of a smart car using PYNQ</font>

<br>
<br>
<br>
    
<center> <font size="6" color="#0086DD">Internet of Things (IoT)</font>
<br>
<br>
<br>

<center> <font size="5" color="#0086DD">Joaquim Morera & Aarón Montiel</font>
<br>
<br>


## 1. Motivation, objectives and materials

### 1.1 Motivation

We want to provide a car manufacturer IoT solutions to integrate within the services that they offer in order to be more competitive and deliver a better value proposition. 

We are developing functionalities that assist the car owners when they are driving their car and also when it is parked and away from the user’s vision. These functionalities have been achieved thanks to the sensorization of the vehicle, as well as the monitoring of different aspects of concern to the user such as temperature, brightness, or human presence around the car.

A voice service to assist the driver is also implemented using Speech Recognition techniques. In addition, the state of the car is reported periodically to a mobile app when the owner can’t be physically aware of it in order to inform him/her if problems are found. Also, this very app powers on and powers off the vehicle without the need of a physical key.


### 1.2 Objetives

The objectives of this project are the following:
- Sensorize and assist accordingly to the needs of a car owner and its vehicle.
- Use a communication system that periodically reports the status of the car.
- Make use of a voice assistant integrating Speech Recognition.


### 1.3 Materials

To carry out this project, we will need the following hardware and software elements. On the hardware level we have used:

- PYNQ-Z2 Development Board  based on Xilinx ZYNQ SoCs [[1]](http://www.pynq.io/board.html). This will be the processor of all tasks.
- Arduino adapter shield [[2]](https://www.seeedstudio.com/Base-Shield-V2.html), in order to handle all the sensors and activators.
- Microphone. A normal microphone with a Jack connector.
- Sensors:
    - Water sensor [[3]](https://www.seeedstudio.com/Grove-Water-Sensor.html).
    - Temperature sensor [[4]](https://www.seeedstudio.com/Grove-Temperature-Sensor.html)
    - Ambient light sensor [[5]](https://www.seeedstudio.com/Grove-Light-Sensor-v1-2-LS06-S-phototransistor.html).
    - Motion sensor [[6]](https://www.seeedstudio.com/Grove-PIR-Motion-Sensor.html).
- Actuators:
    - Buzzer [[7]](https://www.seeedstudio.com/Grove-Buzzer.html).
    - Led stick [[8]](https://www.seeedstudio.com/Grove-RGB-LED-Stick-10-WS2813-Mini.html).

    
At the software level we will use, among other common-use libraries: 
- MQTT [[9]](https://mqtt.org/) technology as a wireless communication protocol between the different devices.
- SpeechRecognition [[10]](https://pypi.org/project/SpeechRecognition/), that provides the ability to convert speech to text.
- GTTS [[11]](https://gtts.readthedocs.io/en/latest/), that converts text to speech.
- MQTT Dash [[12]](https://play.google.com/store/apps/details?id=net.routix.mqttdash&hl=ca&gl=US), that powers the mobile app and handles MQTT communication.

## 2. Necessary installations and libraries

In [ ]:
!pip install SpeechRecognition
!pip install gTTS
!pip install ipython
!pip install paho-mqtt
!apt-get install flac

### 2.1 Checking that the installations have been carried out correctly and importing the necessary libraries.

In [ ]:
try:
    import speech_recognition as sr
except ImportError:
    print("This script requires the the following module installs: \n"
          "1. FLAC : sudo apt-get install flac \n"
          "2. portaudio19-dev : sudo apt-get install portaudio19-dev \n"
          "3. pyaudio : sudo pip3 install pyaudio\n")
    print("Then install speech_recognition module with: \n"
         "sudo pip3 install SpeechRecognition")
try:
    from gtts import gTTS
except ImportError:
    print("This script requires the gtts module \n Install with: sudo "
         "pip3 install gtts")
    
from IPython.display import display, Audio
import os
import re
from time import sleep
import json
import datetime
import paho.mqtt.client as mqtt
import sys
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoSEEEDGroveAdapter
from time import sleep

## 3. Initial Configurations


### 3.1 Setting up the MQTT

In [ ]:
host          = "node02.myqtthub.com"
port          = 1883
clean_session = True
client_id     = "client_pynq"
user_name     = "client_pynq"
password      = "client_pynq"

#Creating logic functions
def on_connect (client, userdata, flags, rc):
    """ Callback called when connection/reconnection is detected """
    print ("Connect %s result is: %s" % (host, rc))

    if rc == 0:
        client.connected_flag = True
        print ("connected OK")
        return
    
    print ("Failed to connect to %s, error was, rc=%s" % rc)
    sys.exit (-1)

def on_message(client, userdata, msg):
    """ Callback called for every PUBLISH received """
    global power
    
    # If button is on
    msg_decoded = str(msg.payload.decode("utf-8"))
    
    if msg_decoded == "1":
        power= "ON"
        metrics = json.dumps({'color': '#86DC3D',
                        "alert_icon": "0",
                        "alert_text": "Driving"})
    
        client.publish("metrics/exchange", metrics)
        client.loop()
        
    elif msg_decoded == "0":
        power = "OFF"
    else:
        pass
    
    return

# Define clientId, host, user and password
client = mqtt.Client (client_id = client_id, clean_session = clean_session)
client.username_pw_set(user_name, password)

# Assigning the funcions
client.on_connect = on_connect
client.on_message = on_message

#### 3.1.1. Function to connect the client

In [ ]:
def ClientConnect():
    client.connect (host, port, keepalive = 60)
    client.connected_flag = False
    while not client.connected_flag:
        client.loop()
        sleep (1)

    # Subscription to the topic "power" that will tell when the car is on or not.
    client.subscribe("power")

### 3.2 Setting up the PYNQ with sensors, actuators and microphone 

In [ ]:
base = BaseOverlay("base.bit")
adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO,
                                   A0='grove_temperature',
                                   A1 = 'grove_pir',
                                   A2 = 'grove_water_sensor',
                                   A3 = 'grove_light', 
                                   D5 = 'grove_servo', 
                                   D6 = 'grove_buzzer',
                                   D4 = 'grove_led_stick')

#Sensors
temp_sensor = adapter.A0
pir_sensor = adapter.A1
water_sensor = adapter.A2
light_sensor = adapter.A3

#Actuators
servo = adapter.D5
buzzer = adapter.D6
ledstick = adapter.D4

#Microphone
r = sr.Recognizer()
pAudio = base.audio
pAudio.set_volume(20)
pAudio.select_microphone()
pAudio.bypass(seconds=5)

## 3. Development of the car's functionalities

The car we have developed has different functionalities depending on whether the user is driving it or it is parked and the owner can’t be physically aware of it. 

### 3.1 Driving Mode

When it is being driven (Driving mode) the following functionalities are available: 

#### 3.1.1 Rain Detector

When rain is detected, the wiper gets activated automatically.


In [ ]:
def RainDetector():
    
    if not water_sensor.is_dry:
        sleep(0.5)
        servo.set_angular_position(180)
        sleep(0.5)
        servo.set_angular_position(0)
    
    return

#### 3.1.2 Headlights Handler

Led lights adapt to ambient light in order to achieve a comfortable sight. When the outdoor light gets darker, the LED headlights get brighter.


In [ ]:
def Headlights():
    
    intensity = light_sensor.get_intensity()/100
    
    id_lights = []
    
    if intensity > 0.8:
        id_lights = [0,9]
        
    elif intensity > 0.6:
        id_lights = [0,9,1,8]
        
    elif intensity > 0.4:
        id_lights = [0,9,1,8,2,7]
        
    elif intensity > 0.2:
        id_lights = [0,9,1,8,2,7,3,6]
    
    else:
        id_lights = [0,9,1,8,2,7,3,6,4,5]
        
    ledstick.clear()     
    for i in id_lights:
        ledstick.set_pixel(i, 0xfffff7)
          
    n_leds = len(id_lights)
    
    ledstick.show() 
    sleep(1)
    return

#### 3.1.3 Parking Assistant

It detects the proximity of objects from behind and helps the driver when parking by sending audio warnings. This functionality is activated when the user asks for it, in our case, when a switch (switch 1) is activated.


In [ ]:
def ParkingAssistant():
    
    intensity = light_sensor.get_intensity()/100
    
    if intensity < 0.7:
        buzzer.play_tone(80, 1)
        sleep(1* intensity/3)
    
    return

#### 3.1.4 Voice Assistant


##### 3.1.4.1 Voice Functions

In [ ]:
def temperature():
    return f'temperature is {temp_sensor.get_temperature()-25:.2f} degree centigrade'

def play_september():
    display(Audio(("data/September_20s.mp3"), autoplay=True))
    sleep(3)

def record_voice(seconds):
    # Record audio
    pAudio.record(seconds)
    pAudio.save("recording_1.wav")

    # Process audio
    harvard = sr.AudioFile('recording_1.wav')
    with harvard as source:
        audio = r.record(source)
    
    # Voice recognizer
    try:
        text_in = r.recognize_google(audio)
        return text_in

    except:
        return  "---------"
    

def t2s(text): #text to sound
    try:
        tts = gTTS(text=text, lang='en', slow=False)
        speech = os.getcwd() + '/output.mp3'
        tts.save(speech)  # save a wav file
        return speech
    except:
        tts = gTTS(text="Sorry try again", lang='en', slow=False)
        speech = os.getcwd() + '/output.mp3'
        tts.save(speech)  # save a wav file
        return speech

def reply(input_text):
    display(Audio(t2s(input_text), autoplay=True))
    sleep(3)

##### 3.1.4.2 Voice Assistant function

When a button is pressed (button 3), a voice assistant responds to the user’s needs. Through the understanding of what the user asks, it provides three different services:

- Temperature handler: When the driver asks for a cooler or hotter ambient, the car detects the temperature and responds accordingly.
- Hour responder: When the driver asks for the actual hour, the voice assistant responds with the exact hour.
- Music Assistant: When the driver asks for music and the car plays a song.


In [ ]:
def VoiceAssistant():
    reply("Hey, what do you need?")
    print("--- Speak ---")
    text_in = record_voice(seconds = 2.5)
    print("Your request: ", text_in)
        
    if re.search('temperature', text_in):
        temp_response = temperature()
        reply(temp_response)
        sleep(0.8)
        reply("Do you want the temperature to go up or down?")
        print("--- Speak ---")
        text_in = record_voice(seconds = 2.5)
        print("Your request: ", text_in)
        
        if re.search('up|Up', text_in):
            reply("Ok! Warming up the car")
            
        elif re.search('down|Down', text_in):
            reply("Ok! Cooling the car")
        else: 
            reply("Sorry! I didn't understand you")
    
    elif re.search('music|Music', text_in):
        play_september()
        
    elif re.search('time|Time', text_in):
        now = datetime.datetime.now()
        hour = now.hour + 1
        mins = now.minute
        reply("It is " + str(hour) + " " + str(mins))
    
    else:
        reply("Sorry, I didn't understand you")

### 3.2 Security Mode 

Once the car is parked, the security mode will be activated, and real-time information about the car's status will be provided. The functionalities of the Security Mode are:



#### 3.2.1 Human Presence Detector

The car detects if there are objects getting closer to it and it responds with an acoustic and luminous alarm, to scare off intruders.

In [ ]:
def HumanDetector():
    
    motion = pir_sensor.motion_detected()
    
    if motion:
    
        ledstick.clear()
        sleep(0.5)
        buzzer.play_tone(80, 200)
        for i in range(10):
            ledstick.set_pixel(i, 0xff0000)
        ledstick.show()    
        buzzer.play_tone(80, 200)
        servo.set_angular_position(180)
        sleep(0.5)
        servo.set_angular_position(0)
    
    return

#### 3.2.2 Metrics Publisher to MQTT

The state of the car is reported periodically to a mobile app to inform the user if problems are found. Specifically, it informs of the weather, temperature and human presence through an alert system.


In [ ]:
def PublishMetrics():

    # Variables
    alert_text = ""
    alert_level = 0
    
    # --------------- Data collection from the sensors
    #Temperature
    temp = round(temp_sensor.get_temperature() - 25, 2)
    if temp < 0 or temp > 40:
        alert_text += "Anormal temperature\n"
        alert_level += 1
    
    #Rain
    if water_sensor.is_dry():
        rain = "0"
    else:
        rain= "1"
        alert_text += "Rain detected\n"
        alert_level += 1

    # Motion
    if pir_sensor.motion_detected():
        alert_text += "Motion detected"
        alert_level += 1

    # --------------- Variables change according to previous
    # Alert icon 
    if alert_level >=1:
        alert_icon = "1"
    else:
        alert_icon = "0"

    # Alert text
    if alert_text == "":
        alert_text = "None so far!"
    
    # General color of the car
    if alert_level == 0:
        color =  '#86DC3D' #green
    elif alert_level >= 2:
        color =  '#ff0000' #red
    else:
        color = '#ffCD01' #yellow

    # --------------- Publish of the metrics
    metrics = json.dumps({'color':color,
                        "rain": rain,
                        "temperature":temp,
                        "alert_icon": alert_icon,
                        "alert_text": alert_text})
    
    client.publish("metrics/exchange", metrics)
    client.loop()
    sleep(3)

    return
    

## 4. Smart car implementation

Below is a usage diagram of our application and the algorithm needed to run it. The diagram shows how to access the different functionalities of the smart car through the activation and deactivation of the available modes (Driving and Security). To do this, we will need to have the PYNQ connected to the internet in order to use the MQTT functionalities that turn the car on or off and see its status in real time. 


![Usage Diagram](./Usage_Diagram.png)

**Algorithm Execution:**

In [ ]:
power = "OFF"
ClientConnect()

#PYNQ ON (is selected by the switch 0) 
while base.switches[0].read() == 1: 
    
    client.loop()
    switch_parking_mode = base.switches[1].read()
    
    #DRIVING MODE (power in MQTT app is OFF)
    if power == "ON": 
        Headlights()
        sleep(0.2)
        RainDetector()
        
        #VOICE ASSISANT (is selected using button 3)
        if base.buttons[3].read() == 1:
            VoiceAssistant()
            
        #PARKING MODE (is selected by the switch 1)
        while switch_parking_mode == 1: 
            ParkingAssistant()
            switch_parking_mode = base.switches[1].read()
        
    else: #SECURITY MODE (power in MQTT app is OFF)
        ledstick.clear()
        HumanDetector()
        PublishMetrics()


ledstick.clear()
client.disconnect()